In [ ]:
import os 

In [ ]:
os.chdir("drive/MyDrive/bmi/")

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.6 MB/s 
     |████████████████████████████████| 7.6 MB 55.7 MB/s 
     |████████████████████████████████| 182 kB 73.0 MB/s 


In [ ]:
import os 
import argparse 
import random 
import pandas as pd
import numpy as np 
import collections 
from tqdm import tqdm 
from datetime import datetime

# PyTorch libraries 
import torch 
import torch.nn as nn 
from torch.utils.data import DataLoader 
from torchvision.datasets import DatasetFolder
from torchvision import models, transforms 

# Hugging Face datasets 
#import datasets 

# Transformers libraries 
from transformers import TrainingArguments, Trainer
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import get_linear_schedule_with_warmup 

# import sklearn models 
# from sklearn.neighbors import KNeighborsClassifier

# simple models
from models import LogisticRegression, BasicCNNModel, DenseCNNModel, BasicCNNCountryModel, ViTCountryModel, ViTMosaiksModel
from SatelliteImageDataset import SatelliteImageDataset, SatelliteImageMetadataDataset, SatelliteImageMosaiksDataset

from sklearn.metrics import confusion_matrix

In [ ]:
RANDOM_SEED = 231 
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

class ImageClassificationCollator:
    def __init__(self, feature_extractor, transforms = False, metadata = False, mosaiks = False): 
        self.feature_extractor = feature_extractor
        self.transforms = transforms 
        self.metadata = metadata
        self.mosaiks = mosaiks

    def __call__(self, batch):
        if self.transforms: 
            transformed = [self.feature_extractor(x[0].cpu().detach().numpy()) for x in batch]
            encodings = {"pixel_values":torch.stack(transformed)}
        else: 
            encodings = self.feature_extractor([x[0] for x in batch], return_tensors='pt')   
        encodings['labels'] = torch.tensor([x[1] for x in batch],  dtype=torch.long)
        
        if self.metadata: 
            if "country" in self.metadata:
                encodings['country'] = torch.tensor([x[2] for x in batch])
        elif self.mosaiks: 
            encodings['mosaiks_features'] = torch.tensor(np.array([x[2] for x in batch]), dtype = torch.float32)

        return encodings

# create model and collator
def create_model_and_collator(args, model_name, metadata = None, cnt_id_map = None):

    if metadata:
        feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
        collator = ImageClassificationCollator(feature_extractor, metadata=metadata)
        collators = (collator, collator)
        if model_name in ['basic_cnn']:
            model = BasicCNNCountryModel(n_classes=CLASSES, cnt_id_map = cnt_id_map, num_country_embeddings=len(cnt_id_map))
        elif model_name == "ViT":
            model = ViTCountryModel(n_classes=CLASSES, cnt_id_map = cnt_id_map, num_country_embeddings=len(cnt_id_map))
    elif args['mosaiks']: 
        feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
        collator = ImageClassificationCollator(feature_extractor, mosaiks=args['mosaiks'])
        collators = (collator, collator)
        model = ViTMosaiksModel(n_classes=CLASSES, mosaiks_dim = 3999)
    elif model_name == "ViT":
        feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
        collator = ImageClassificationCollator(feature_extractor)
        collators = (collator, collator)
        model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=CLASSES)

    elif model_name in ['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet']:
        # note all models expect image of (3, 224, 224)

        train_data_transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomResizedCrop(224), # i.e. want 224 by 224 
            transforms.RandomHorizontalFlip(),  
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        val_data_transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(224), # i.e. want 224 by 224 
            transforms.CenterCrop(224), 
            transforms.ToTensor(), 
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        train_collator = ImageClassificationCollator(train_data_transforms, transforms=True)
        val_collator = ImageClassificationCollator(val_data_transforms, transforms=True)

        collators = (train_collator, val_collator)

        if model_name == 'resnet':
            model = models.resnet18(pretrained=True)
            model.fc = nn.Linear(model.fc.in_features, CLASSES)

        elif model_name == 'alexnet':
            model = models.alexnet(pretrained=True)
            model.classifier[6] = nn.Linear(model.classifier[6].in_features, CLASSES)

        elif model_name == 'vgg':
            model = models.vgg11_bn(pretrained=True)
            model.classifier[6] = nn.Linear(model.classifier[6].in_features, CLASSES)

        elif model_name == 'squeezenet': 
            model = models.squeezenet1_0(pretrained=True)
            model.classifier[1] = nn.Conv2d(512, CLASSES, kernel_size=1, stride=1)
            model.num_classes = CLASSES

        else: 
            # dense net 
            model = models.densenet121(pretrained=True)
            model.classifier = nn.Linear(model.classifier.in_features, CLASSES) 

    elif model_name in ['basic_cnn', 'basic_cnn_novit', 'dense_cnn', 'logistic_regression']:
        # ADD IN transforms though feature extractor might be easier 
        if "novit" in model_name:
            train_data_transforms = transforms.Compose([
                transforms.ToPILImage(),
                transforms.RandomResizedCrop(224), # i.e. want 224 by 224 
                transforms.RandomHorizontalFlip(),  
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
            val_data_transforms = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(224), # i.e. want 224 by 224 
                transforms.CenterCrop(224),
                transforms.ToTensor(), 
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
            train_collator = ImageClassificationCollator(train_data_transforms, transforms=True)
            val_collator = ImageClassificationCollator(val_data_transforms, transforms=True)

            collators = (train_collator, val_collator)
        else: 
            feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
            collator = ImageClassificationCollator(feature_extractor)
            collators = (collator, collator)
        # TODO: add support for model!
        if model_name == "logistic_regression":
            model = LogisticRegression(n_classes=CLASSES)
        elif "basic_cnn" in model_name:
            model = BasicCNNModel(n_classes=CLASSES)
        elif model_name == "dense_cnn":
            model = DenseCNNModel(n_classes=CLASSES)

    else: 
        raise NotImplementedError

    print(f'Model name: {model_name}')

    return collators, model 


def create_dataset(args, collator_fns, metadata = None, cnt_id_map = None, val_split = 0.15):

    def npy_loader(path):
        sample = torch.from_numpy(np.load(path))
        return sample 
    
    # load in dataset frmom directory 
    if metadata:
        dataset = SatelliteImageMetadataDataset(
            root = args['data_dir'], 
            csv_path = args['csv_file'], 
            outcome = args['outcome'], 
            loader = npy_loader
        )
    elif args['mosaiks']: 
        dataset = SatelliteImageMosaiksDataset(
            root = args['data_dir'], 
            csv_path = args['csv_file'], 
            outcome = args['outcome'], 
            mosaiks_csv_path = args['mosaiks_csv_file'], 
            loader = npy_loader
        )
    else:
        dataset = SatelliteImageDataset(
            root = args['data_dir'], 
            csv_path = args['csv_file'],
            outcome = args['outcome'], 
            loader = npy_loader
        )

    # IDEALLY we would like same sampling...

    # split up into train val data 
    if os.path.isfile("indices_perm2.npy"):
        indices = np.load("indices_perm2.npy") 
    else:
        indices = torch.randperm(len(dataset)).tolist()
        np.save("indices_perm2.npy", indices)
    
    n_val = int(np.floor(len(indices) * val_split))
    train_ds = torch.utils.data.Subset(dataset, indices[:-n_val])
    val_ds = torch.utils.data.Subset(dataset, indices[-n_val:])

    train_dl = DataLoader(train_ds, batch_size=args['batch_size'], collate_fn=collator_fns[0], shuffle = 1)
    val_dl = DataLoader(val_ds, batch_size=args['batch_size'], collate_fn=collator_fns[1], shuffle=0)

    return [train_dl, val_dl]

def dataset_statistics(args, dataset_loader):
    label_stats = collections.Counter()
    for i, batch in enumerate(dataset_loader):
        inputs, labels = batch['pixel_values'], batch['labels']
        labels = labels.cpu().numpy().flatten()
        label_stats += collections.Counter(labels)
    return label_stats


def measure_accuracy(outputs, labels):
    preds = np.argmax(outputs, axis = 1).flatten()
    labels = labels.flatten()
    correct = np.sum(preds == labels)
    c_matrix = confusion_matrix(labels, preds, labels=CLASS_NAMES)
    return correct, len(labels), c_matrix 

def validation(args, val_loader, model, criterion, metadata, device, name = 'Validation', write_file=None):

    model.eval()
    total_loss = 0. 
    total_correct = 0 
    total_sample = 0 
    total_confusion = np.zeros((CLASSES, CLASSES))

    for i, batch in enumerate(tqdm(val_loader)):
        inputs, labels = batch['pixel_values'], batch['labels'] 
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            if metadata: 
                country = batch['country'].to(device)
                outputs = model(inputs, country)
            elif args['mosaiks']: 
                mosaiks_features = batch['mosaiks_features'].to(device)
                outputs = model(inputs, mosaiks_features)
            elif args['model_name'] in [
            'basic_cnn', 'basic_cnn_novit', 'dense_cnn', 'logistic_regression',
            'resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet'
            ]:
                outputs = model(inputs)
            else: 
                outputs = model(inputs)['logits'] 

        loss = criterion(outputs, labels)

        logits = outputs 
        total_loss += loss.cpu().item()

        correct_n, sample_n, c_matrix = measure_accuracy(logits.cpu().numpy(), labels.cpu().numpy())
        total_correct += correct_n 
        total_sample += sample_n 
        total_confusion += c_matrix 

    print(f'*** Accuracy on the {name} set: {total_correct/total_sample}')
    print(f'*** Weighted accuracy on the {name} set: {np.mean( np.diag(total_confusion) / np.sum(total_confusion, 1) )}')
    print(f'*** Confusion matrix:\n{total_confusion}')
    if write_file:
        write_file.write(f'*** Accuracy on the {name} set: {total_correct/total_sample}\n')
        write_file.write(f'*** Weighted accuracy on the {name} set: {np.mean( np.diag(total_confusion) / np.sum(total_confusion, 1) )}')
        write_file.write(f'*** Confusion matrix:\n{total_confusion}\n')

    return total_loss, float(total_correct / total_sample) * 100



def train(args, data_loaders, epoch_n, model, optim, scheduler, criterion, metadata, device, write_file=None):
    print("\n>>> Training starts...")

    if write_file: 
        write_file.write("\n>>> Training starts...")

    model.train()

    best_val_acc = 0
    for epoch in range(epoch_n):
        print("*** Epoch:", epoch)
        total_train_loss = 0. 
        total_correct = 0
        total_sample = 0

        for i, batch in enumerate(tqdm(data_loaders[0])): 
            optim.zero_grad()
            inputs, labels = batch['pixel_values'], batch['labels'] 
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            # forward pass 
            if metadata: 
                country = batch['country'].to(device)
                outputs = model(inputs, country)
            elif args['mosaiks']: 
                mosaiks_features = batch['mosaiks_features'].to(device)
                outputs = model(inputs, mosaiks_features)
            elif args['model_name'] in [
                'basic_cnn', 'basic_cnn_novit', 'dense_cnn', 'logistic_regression', 
                'resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet'
            ]:
                outputs = model(inputs)
            else: 
                outputs = model(inputs)['logits']

            loss = criterion(outputs, labels)
            logits = outputs
            correct_n, sample_n, c_matrix = measure_accuracy(logits.cpu().detach().numpy(), labels.cpu().detach().numpy())
            total_correct += correct_n 
            total_sample += sample_n 

            total_train_loss += loss.cpu().item()

            # backward pass 
            loss.backward()
            optim.step()

            if scheduler: scheduler.step()

            if i % args['val_every'] == 0: 
                print(f'*** Average Loss: {total_train_loss / (i+1)}')
                print(f'*** Running accuracy on the train set: {total_correct/total_sample}')
                if write_file:
                    write_file.write(f'\nEpoch: {epoch}, Step: {i}\n')
                    write_file.write(f'*** Loss: {loss}\n')
                    write_file.write(f'*** Running accuracy on the train set: {total_correct/total_sample}\n')

                _, val_acc = validation(args, data_loaders[1], model, criterion, metadata, device, write_file=write_file)

                model.train()

                if best_val_acc < val_acc: 
                    best_val_acc = val_acc 

                    if args['save_path']:
                        if args['mosaiks']:
                          with open(args['save_path'] + ".pkl", "wb") as f:
                            pickle.dump(model, f)
                        elif args['model_name'] in ['ViT']:
                            model.save_pretrained(args['save_path'])
                        else: 
                            torch.save(model.state_dict(), args['save_path'])



In [ ]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F

from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers import ViTForImageClassification, ViTModel, ViTConfig

class ViTMosaiksModel(nn.Module):

	def __init__(self, n_classes, mosaiks_dim = 64, mlp_dim = 128):
		
		super().__init__()

		self.n_classes = n_classes
		# applies pooling layer 
		configuration = ViTConfig()
		self.model = ViTModel(configuration).from_pretrained('google/vit-base-patch16-224-in21k')
		
		hidden_dim = 768
		self.mlp = nn.Sequential(
            nn.Linear(hidden_dim + mosaiks_dim, mlp_dim),
            nn.ReLU(),
            nn.Linear(mlp_dim, mlp_dim),
            nn.ReLU(),            
            nn.Linear(mlp_dim, self.n_classes)
        )
	
	def forward(self, X, mosaiks_features):
		device = 'cuda' if torch.cuda.is_available() else 'cpu'
		model_out = self.model(X)['pooler_output']
		mosaiks_features = mosaiks_features.to(device)
		concat_output = torch.cat((model_out, mosaiks_features), dim=1)
		logits = self.mlp(concat_output)

		return logits

In [ ]:
args = {
  'data_dir': 'west_africa',
  'csv_file': 'west_africa_df', 
  'mosaiks_csv_file': 'west_africa_mosaiks_feats',
  'mosaiks': False, 
  'outcome': 'Mean_BMI_bin',
  'model_name':'ViT',

  'val_every': 200, 
  'batch_size': 64, 
  'lr':2e-5, 
  'eps':1e-8
}

# set device to GPU if possible
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Number of classes 
CLASSES = 3
CLASS_NAMES = [i for i in range(CLASSES)]

epoch_n = 10

# read df 
df = pd.read_csv(args['csv_file'] + ".csv")

# if args.metadata and args.mosaiks: 
#     raise NotImplementedError("Functionality for both mosaiks and metadata has not been implemented yet.")

cnt_id_map = None
metadata = None
# if args.metadata: 
#     metadata = ["country"]
#     unique_countries = list(set(df["country"]))
#     unique_countries_int = [int(str(ord(c[0])) + str(ord(c[1]))) for c in unique_countries]
#     cnt_id_map = {float(v):k for k, v in enumerate(set(unique_countries_int))}

# if filename is None: 
#     filename = f'./results/{args.model_name}/{datetime.now()}.txt'

write_file = None
# write_file = open(filename, "w")

# if write_file:
#     write_file.write(f'*** args: {args}\n\n')

# create model 
collators, model = create_model_and_collator(
    args = args, 
    model_name = args['model_name'], 
    metadata=metadata, cnt_id_map = cnt_id_map

)
model.to(device)

In [ ]:
# load data 
data_loaders = create_dataset(
    args = args, collator_fns = collators, metadata = metadata, cnt_id_map = cnt_id_map
)

# train_label_stats = dataset_statistics(args, data_loaders[0])
# val_label_stats = dataset_statistics(args, data_loaders[1])
# print(f'*** Training set label statistics: {train_label_stats}')
# print(f'*** Validation set label statistics: {val_label_stats}')

# if write_file:
#     write_file.write(f'*** Training set label statistics: {train_label_stats}')
#     write_file.write(f'*** Validation set label statistics: {val_label_stats}')	


# if args.model_name in ['logistic_regression', 'basic_cnn', 'dense_cnn']:
#     optim = torch.optim.Adam(params = model.parameters())
# else: 
optim = torch.optim.AdamW(params=model.parameters(), lr=args['lr'], eps=args['eps'])

total_steps = len(data_loaders[0]) * epoch_n 
scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=0, num_training_steps = total_steps)

# get class weights 
class_weights = 1 - df[args['outcome']].value_counts(normalize=True).sort_index()
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

if write_file: 
    write_file.write(f'\nModel:\n {model}\nOptimizer:{optim}\n')

In [ ]:
import pickle

In [ ]:
args['save_path'] = 'ViT_mosiaks_model.pkl'

In [ ]:
args['lr'] = 1e-5

In [ ]:
train(args, data_loaders, epoch_n, model, optim, scheduler, criterion, metadata, device, write_file)

if write_file:
    write_file.close()


>>> Training starts...
*** Epoch: 0


  0%|          | 0/97 [00:00<?, ?it/s]

*** Average Loss: 1.119106650352478
*** Running accuracy on the train set: 0.078125



100%|██████████| 17/17 [05:39<00:00, 19.94s/it]


*** Accuracy on the Validation set: 0.17988929889298894
*** Weighted accuracy on the Validation set: 0.36829457364341084
*** Confusion matrix:
[[ 57.   0.  43.]
 [365.   0. 361.]
 [120.   0. 138.]]


...

  0%|          | 0/97 [00:00<?, ?it/s]

*** Average Loss: 0.171013742685318
*** Running accuracy on the train set: 1.0



100%|██████████| 17/17 [00:29<00:00,  1.76s/it]


*** Accuracy on the Validation set: 0.6494464944649446
*** Weighted accuracy on the Validation set: 0.4814914472419758
*** Confusion matrix:
[[ 20.  76.   4.]
 [ 56. 563. 107.]
 [  3. 134. 121.]]


100%|██████████| 97/97 [05:28<00:00,  3.39s/it]


In [ ]:
train(args, data_loaders, epoch_n, model, optim, scheduler, criterion, metadata, device, write_file)

if write_file:
    write_file.close()


>>> Training starts...
*** Epoch: 0


  0%|          | 0/97 [00:00<?, ?it/s]

*** Average Loss: 1.1114825010299683
*** Running accuracy on the train set: 0.328125



100%|██████████| 17/17 [00:51<00:00,  3.01s/it]


*** Accuracy on the Validation set: 0.3284132841328413
*** Weighted accuracy on the Validation set: 0.4130311455612661
*** Confusion matrix:
[[ 48.  34.  38.]
 [165. 152. 398.]
 [ 43.  50. 156.]]


...

*** Average Loss: 0.24936728179454803
*** Running accuracy on the train set: 0.9375



  1%|          | 1/97 [00:28<45:55, 28.70s/it]

*** Accuracy on the Validation set: 0.6605166051660517
*** Weighted accuracy on the Validation set: 0.49375005850909465
*** Confusion matrix:
[[ 39.  79.   2.]
 [ 52. 597.  66.]
 [  5. 164.  80.]]


  3%|▎         | 3/97 [00:35<18:39, 11.91s/it]


KeyboardInterrupt: ignored

In [ ]:
## MOSAIKS RESULTS

In [ ]:
df = pd.read_csv("west_africa_df.csv")

In [ ]:
indices = np.load("indices_perm2.npy")
n_val =int(np.floor(len(indices) * 0.15))

In [ ]:
train_dhsids = pd.Series(np.array(
    data_loaders[0].dataset.dataset.image_paths
)[indices[:-n_val]]).apply(lambda x: x.split('.')[0])
train_df = df.loc[df["DHSID"].isin(train_dhsids)]
val_df = df.loc[~df["DHSID"].isin(train_dhsids)]

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
mosaiks_df = pd.read_csv("west_africa_mosaiks_feats.csv")

In [ ]:
train_mosaiks = mosaiks_df.loc[mosaiks_df['DHSID'].isin(train_dhsids)]
val_mosaiks = mosaiks_df.loc[~mosaiks_df['DHSID'].isin(train_dhsids)]

In [ ]:
mosaiks_feats = [" ." + str(i + 1) for i in range(3999)]

In [ ]:
train_mosaiks_fts = train_mosaiks[mosaiks_feats]
val_mosaiks_fts = val_mosaiks[mosaiks_feats]

In [ ]:
# MEAN BMI BIN
lr = LogisticRegression(random_state = 231, class_weight = 'balanced', max_iter = 1000, tol = 0.001)
lr.fit(train_mosaiks_fts, train_df["Mean_BMI_bin"])
preds = lr.predict(val_mosaiks_fts)
cm = confusion_matrix(val_df["Mean_BMI_bin"], preds, labels=CLASS_NAMES)
np.mean(
    np.diag(cm) / np.sum(cm, 1)
)

In [ ]:
cm = confusion_matrix(val_df["Mean_BMI_bin"], preds, labels=CLASS_NAMES)
np.mean(
    np.diag(cm) / np.sum(cm, 1)
)

0.46771063281824876

In [ ]:
# UNDER 5 BIN
u5_lr = LogisticRegression(random_state = 231, class_weight = 'balanced', max_iter = 2000)
u5_lr.fit(train_mosaiks_fts, train_df["Under5_Mortality_Rate_bin"])
preds = u5_lr.predict(val_mosaiks_fts)
cm = confusion_matrix(val_df["Under5_Mortality_Rate_bin"], preds, labels=CLASS_NAMES)
np.mean(
    np.diag(cm) / np.sum(cm, 1)
)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.3884044720511475

In [ ]:
# UNDER 5 BIN QUINT
u5q_lr = LogisticRegression(random_state = 231, class_weight = 'balanced', max_iter = 4000)
u5q_lr.fit(train_mosaiks_fts, train_df["Under5_Mortality_Rate_bin_quint"])
preds = u5q_lr.predict(val_mosaiks_fts)
cm = confusion_matrix(val_df["Under5_Mortality_Rate_bin_quint"], preds)
np.mean(
    np.diag(cm) / np.sum(cm, 1)
)

0.24587755444298157

In [ ]:
# MEAN BMI QUINT
bmiq_lr = LogisticRegression(random_state = 231, class_weight = 'balanced', max_iter = 4000)
bmiq_lr.fit(train_mosaiks_fts, train_df["Mean_BMI_bin_quint"])
preds = bmiq_lr.predict(val_mosaiks_fts)
cm = confusion_matrix(val_df["Mean_BMI_bin_quint"], preds)
np.mean(
    np.diag(cm) / np.sum(cm, 1)
)

0.25441700648433796

In [ ]:
# UNDER 5 BIN QUINT
u5q_lr = LogisticRegression(random_state = 231, class_weight = 'balanced', max_iter = 4000, C = 0.1)
u5q_lr.fit(train_mosaiks_fts, train_df["Under5_Mortality_Rate_bin_quint"])
preds = u5q_lr.predict(val_mosaiks_fts)
cm = confusion_matrix(val_df["Under5_Mortality_Rate_bin_quint"], preds)
np.mean(
    np.diag(cm) / np.sum(cm, 1)
)

0.24932198275131584